In [3]:
import numpy as np
import pandas as pd

# ----------------------------
# MERGED INPUT SECTION
# ----------------------------

months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

# Input data from user
actual_kwh = [3500, 6000, 2000, 1000, 3639.09091, 100, 930, 1500, 5000, 5500, 6500, 8000] #User Input Consumption (kWh per month)
days_per_month = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31] #Hard Code
climate_index_h = [1.01, 0.93, 0.83, 0.79, 0.65, 0.91, 0.64, 0.72, 0.97, 1.02, 1.00, 1.02] #Call form our town selection function in cedarpumps.com

# Technical constants
motor_kw = 11
solar_kw = 15.54
drive_min_pct = 0.35
loss_factor_hourly = 0.0 #Ignore and hard code to zero
loss_factor_monthly = 0.20
kwh_cost = 2.6

# Solar production profile (Veichi Tech!B33:B45)
sunshine_percent = np.array([0, 0.03, 0.32, 0.5, 0.75, 0.9, 1.0, 0.99, 0.85, 0.65, 0.3, 0.01, 0])

# ----------------------------
# CALCULATION BLOCK 1: Pump Hours Per Day
# ----------------------------

kwh_per_day = [actual_kwh[i] / days_per_month[i] for i in range(12)]
monthly_pump_hours = {months[i]: round(kwh_per_day[i] / motor_kw, 9) for i in range(12)}

# ----------------------------
# CALCULATION BLOCK 2: Hourly Yield (Veichi Hourly Block)
# ----------------------------

sizing_factor = solar_kw / motor_kw
drive_output = np.minimum(sunshine_percent * sizing_factor, 1.0)
F = np.full_like(drive_output, motor_kw * 1000)
G = np.where(drive_output >= drive_min_pct, drive_output * F, 0)
H = np.where(G > 0, G * (1 - loss_factor_hourly), 0)
I = H / F

def build_excel_matched_distribution_fixed(total_hours):
    spread = np.zeros(13)
    center = 6
    if total_hours <= 1:
        spread[center] = total_hours
        return spread
    spread[center] = 1
    used = 1
    offset = 1
    while used + 2 <= total_hours and offset <= 6:
        spread[center - offset] = 1
        spread[center + offset] = 1
        used += 2
        offset += 1
    remaining = total_hours - used
    if remaining > 0 and offset <= 6:
        spread[center - offset] = remaining / 2
        spread[center + offset] = remaining / 2
    return spread

# Print the spread function results per month
print("\nHourly Spread Matrix (13 hours, per month):")
print("=" * 50)
for month in months:
    spread = build_excel_matched_distribution_fixed(monthly_pump_hours[month])
    spread_str = ", ".join(f"{v:.3f}" for v in spread)
    print(f"{month}: [{spread_str}]")

# Calculate hourly yield
hourly_yield_j = []
for month in months:
    spread = build_excel_matched_distribution_fixed(monthly_pump_hours[month])
    output_kwh = float(np.sum(spread * I))
    hourly_yield_j.append(round(output_kwh, 9))

# ----------------------------
# CALCULATION BLOCK 3: Final Monthly and Yearly Savings
# ----------------------------

results = []
for i in range(12):
    month = months[i]
    actual = actual_kwh[i]
    days = days_per_month[i]
    j = hourly_yield_j[i]
    h = climate_index_h[i]

    k = round(j * h, 9)
    expenditure = round(actual * kwh_cost, 2)
    kwh_day = actual / days
    pump_hours_day = kwh_day / motor_kw
    theoretical_savings = round(k * kwh_cost * motor_kw * days, 10)
    capped_savings = round(min(theoretical_savings, expenditure), 10)
    actual_savings = round(capped_savings * (1 - loss_factor_monthly), 10)

    results.append({
        "Month": month,
        "Pump Hours/Day": round(pump_hours_day, 9),
        "Hourly Yield (J)": j,
        "Yield With Climate (K)": k,
        "Expenditure (R)": expenditure,
        "Theoretical Savings": theoretical_savings,
        "Capped Savings": capped_savings,
        "Actual Savings (L)": actual_savings
    })

final_df = pd.DataFrame(results)
total_savings = round(sum(row["Actual Savings (L)"] for row in results), 2)

# Display final results
print("\nFinal Yearly Savings with Merged Inputs")
print("=" * 50)
print(final_df.to_string(index=False))
print("=" * 50)
print(f"Total Actual Yearly Savings: R {total_savings:,.2f}")



Hourly Spread Matrix (13 hours, per month):
Jan: [0.000, 0.632, 1.000, 1.000, 1.000, 1.000, 1.000, 1.000, 1.000, 1.000, 1.000, 0.632, 0.000]
Feb: [1.000, 1.000, 1.000, 1.000, 1.000, 1.000, 1.000, 1.000, 1.000, 1.000, 1.000, 1.000, 1.000]
Mar: [0.000, 0.000, 0.000, 0.433, 1.000, 1.000, 1.000, 1.000, 1.000, 0.433, 0.000, 0.000, 0.000]
Apr: [0.000, 0.000, 0.000, 0.000, 0.015, 1.000, 1.000, 1.000, 0.015, 0.000, 0.000, 0.000, 0.000]
May: [0.000, 0.836, 1.000, 1.000, 1.000, 1.000, 1.000, 1.000, 1.000, 1.000, 1.000, 0.836, 0.000]
Jun: [0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.303, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000]
Jul: [0.000, 0.000, 0.000, 0.000, 0.000, 0.864, 1.000, 0.864, 0.000, 0.000, 0.000, 0.000, 0.000]
Aug: [0.000, 0.000, 0.000, 0.000, 0.699, 1.000, 1.000, 1.000, 0.699, 0.000, 0.000, 0.000, 0.000]
Sep: [1.000, 1.000, 1.000, 1.000, 1.000, 1.000, 1.000, 1.000, 1.000, 1.000, 1.000, 1.000, 1.000]
Oct: [1.000, 1.000, 1.000, 1.000, 1.000, 1.000, 1.000, 1.000, 1.000, 1.000, 1.000,